# 0. Install and Import Dependencies

In [23]:
import mediapipe as mp
import cv2
import numpy as np
from math import atan2, degrees

In [24]:
mp_drawing = mp.solutions.drawing_utils # helpers for drawing
mp_pose = mp.solutions.pose

In [25]:
VIDEO_PATH = 'bicep-curl.mp4'
EXPORT_PATH = 'coords1.csv'
MODEL_PATH = 'bicep-curl.pkl'

# X. Make Some Detections

In [19]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    try:
        while cap.isOpened():
            ret, image = cap.read()

            if ret is False:
                print("Error Camera Not Found.")
            if image is None:
                print("Ignoring empty camera frame.")
                break

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detections
            results = pose.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                             thickness= 2,
                                                             circle_radius= 4),
                                      mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                             thickness= 2,
                                                             circle_radius= 2)
                                     )
            
            # test code
            
            test = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            test.insert(0, 0)

            left_shoulder = (test[45], test[46])
            right_shoulder = (test[49], test[50])
            left_elbow = (test[53], test[54])
            right_elbow = (test[57], test[58])
            left_wrist = (test[61], test[62])
            right_wrist = (test[65], test[66])
            left_hip = (test[93], test[94])
            right_hip = (test[97], test[98])
            left_knee = (test[101], test[102])
            right_knee = (test[105], test[106])

            left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)

            left_lean_ang = angle(left_shoulder, left_hip, left_knee)
            right_lean_ang = angle(right_shoulder, right_hip, right_knee)

            left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
            right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)

            image = draw_text(image, f'Elbow {int(right_elbow_ang)}', (50, 100))
            image = draw_text(image, f'Lean {int(right_lean_ang)}', (50, 200))
            image = draw_text(image, f'Shoulder {int(right_shoulder_ang)}', (50, 150))
            
            # end test code

            cv2.imshow('LUCIO', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    except Exception as e:
        cap.release()
        cv2.destroyAllWindows()
            
cap.release()
cv2.destroyAllWindows()

# 1. Save Video

In [5]:
cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('bicep-curl.mp4', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()

    try:
        cv2.imshow('Bicep-Curl', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV

In [26]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [27]:
landmarks = ['class']
for val in range(0, 32 +1):
    landmarks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']
landmarks += ['left_elbow_ang', 'right_elbow_ang', 
              'left_lean_ang', 'right_lean_ang', 
              'left_shoulder_ang', 'right_shoulder_ang',
              'left_elbow_hip_dist', 'right_elbow_hip_dist']

In [28]:
with open(EXPORT_PATH, mode= 'w', newline= '') as f:
    csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [29]:
def angle_zibby(p1, p2, p3):
    # x1, y1 = p1
    # x2, y2 = p2
    # x3, y3 = p3
    # deg1 = (360 + degrees(atan2(x1 - x2, y1 - y2))) % 360
    # deg2 = (360 + degrees(atan2(x3 - x2, y3 - y2))) % 360
    # return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)

    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3

    v21 = (x1 - x2, y1 - y2)
    v23 = (x3 - x2, y3 - y2)

    dot = v21[0] * v23[0] + v21[1] * v23[1]
    det = v21[0] * v23[1] - v21[1] * v23[0]

    theta = np.rad2deg(np.arctan2(det, dot))

    return theta

In [30]:
def angle(a, b, c):
    """ Calculate angle between two lines """
    if(a == (0,0) or b == (0,0) or c == (0,0)):
      return 0
    
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # if angle > 180.0:
    #     angle = 360 - angle
        
    return angle

In [31]:
def dist_x(a, b):
    return (b[0] - a[0]) * 100

In [32]:
def draw_text(image, text, pos):
    """ Draw a intersection point in an image """
    cv2.putText(image,
                text,
                org= pos,
                fontFace= cv2.FONT_HERSHEY_PLAIN,
                fontScale= 1.5,
                color= (0, 255, 0),
                thickness= 3)

    return image

In [33]:
def dist_xy(a, b):
    diff_point1 = (a[0] - b[0]) ** 2
    diff_point2 = (a[1] - b[1]) ** 2
    return ((diff_point1 + diff_point2) ** 0.5) * 100

In [34]:
landmarks

['class',
 'x0',
 'y0',
 'z0',
 'v0',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32'

In [35]:
keypoints = [(res.x, res.y) for res in results.pose_landmarks.landmark]
keypoints

[(0.5570637583732605, 0.628878653049469),
 (0.5797590017318726, 0.5590665936470032),
 (0.5971391797065735, 0.5579078197479248),
 (0.6135715246200562, 0.5575447082519531),
 (0.5221171379089355, 0.5564955472946167),
 (0.49999865889549255, 0.554534912109375),
 (0.48150548338890076, 0.5528467893600464),
 (0.6435662508010864, 0.567499041557312),
 (0.4508713185787201, 0.5561506748199463),
 (0.5888267755508423, 0.6956232190132141),
 (0.5193783044815063, 0.7022839188575745),
 (0.8078810572624207, 0.8345028162002563),
 (0.2793850302696228, 0.8437455892562866),
 (0.9526971578598022, 1.0043431520462036),
 (0.06464941799640656, 1.0619785785675049),
 (1.011090636253357, 1.5160621404647827),
 (0.07516276091337204, 1.5253087282180786),
 (1.0328980684280396, 1.6331897974014282),
 (0.07192496955394745, 1.635858416557312),
 (0.984448254108429, 1.6311733722686768),
 (0.12098530679941177, 1.607913613319397),
 (0.9658910632133484, 1.593008041381836),
 (0.13268744945526123, 1.5727975368499756),
 (0.70658910

In [37]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        
        # keypoint == keypoint * 4 + 1 (due to zero indentation and action)
        left_shoulder = (keypoints[45], keypoints[46])
        right_shoulder = (keypoints[49], keypoints[50])
        left_elbow = (keypoints[53], keypoints[54])
        right_elbow = (keypoints[57], keypoints[58])
        left_wrist = (keypoints[61], keypoints[62])
        right_wrist = (keypoints[65], keypoints[66])
        left_hip = (keypoints[93], keypoints[94])
        right_hip = (keypoints[97], keypoints[98])
        left_knee = (keypoints[101], keypoints[102])
        right_knee = (keypoints[105], keypoints[106])
        
        left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)
        
        left_lean_ang = angle(left_shoulder, left_hip, left_knee)
        right_lean_ang = angle(right_shoulder, right_hip, right_knee)
        
        left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
        right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)
        
        left_elbow_hip_dist = dist_x(left_hip, left_elbow)
        right_elbow_hip_dist = dist_x(right_hip, right_elbow)
        
        keypoints.append(left_elbow_ang)
        keypoints.append(right_elbow_ang)
        keypoints.append(left_lean_ang)
        keypoints.append(right_lean_ang)
        keypoints.append(left_shoulder_ang)
        keypoints.append(right_shoulder_ang)
        
        keypoints.append(left_elbow_hip_dist)
        keypoints.append(right_elbow_hip_dist)
                
        with open(EXPORT_PATH, mode= 'a', newline= '') as f:
            csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
            
        return [f'ElbowAng(R): {int(right_elbow_ang)}', f'LeanAng(R): {int(right_lean_ang)}', f'ShoulderAng(R): {int(right_shoulder_ang)}', f'ElbowDist(R): {int(right_elbow_hip_dist)}']
            
    except Exception as e:
        print(f'Exception: {e}')
        pass

In [38]:
cap = cv2.VideoCapture(VIDEO_PATH)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    right_elbow_ang = 0
    right_lean_ang = 0
    right_shoulder_ang = 0
    right_elbow_hip_dist = 0
    features = [f'ElbowAng(R): {right_elbow_ang}', f'LeanAng(R): {right_lean_ang}', f'ShoulderAng(R): {right_shoulder_ang}', f'ElbowDist(R): {right_elbow_hip_dist}']
    
    try:
        while cap.isOpened():
            ret, image = cap.read()

            if ret is False:
                print("Error File Not Found.")
            if image is None:
                print("Ignoring empty camera frame.")
                break     

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detections
            ## results = pose.process(image)
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                             thickness= 2,
                                                             circle_radius= 4),
                                      mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                             thickness= 2,
                                                             circle_radius= 2)
                                     )

            k = cv2.waitKey(1)
            if k == ord('a'):
                features = export_landmark(results, 'correct')
            elif k == ord('s'):
                features = export_landmark(results, 'error1')
            elif k ==ord('d'):
                features = export_landmark(results, 'error2')

            image = draw_text(image, str(features[0]), (40, 550))
            image = draw_text(image, str(features[1]), (40, 600))
            image = draw_text(image, str(features[2]), (40, 575))
            image = draw_text(image, str(features[3]), (40, 625))
            
            cv2.imshow('Bicep Training', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
    except Exception as e:
        print(f'error {e}')
        cap.release()
        cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()

Error File Not Found.
Ignoring empty camera frame.


# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [62]:
df = pd.read_csv(EXPORT_PATH)

In [63]:
df.head()

,class,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z32,v32,left_elbow_ang,right_elbow_ang,left_lean_ang,right_lean_ang,left_shoulder_ang,right_shoulder_ang,left_elbow_hip_dist,right_elbow_hip_dist
0,correct,0.404247,0.818962,0.455553,0.999754,0.409771,0.828992,0.500907,0.999852,0.408371,...,-0.727213,0.824450,41.644140,162.717884,224.053782,221.801313,22.740362,232.382027,-7.514912,-11.038500
1,correct,0.395100,0.814538,0.459111,0.999775,0.404681,0.828165,0.504184,0.999865,0.403966,...,-0.731567,0.830776,33.426070,172.801361,226.882292,221.880520,6.995678,242.880519,-6.945130,-10.509744
2,correct,0.388115,0.810215,0.461728,0.999794,0.399152,0.826347,0.506682,0.999877,0.399012,...,-0.768031,0.836679,38.467783,121.583713,228.088596,221.279565,13.575731,200.808850,-6.972346,-9.009850
3,correct,0.385210,0.805840,0.463203,0.999813,0.395307,0.822718,0.508092,0.999889,0.395182,...,-0.823461,0.843193,38.201204,81.985306,229.768505,220.880685,18.854596,162.619326,-7.201338,-7.628453
4,correct,0.386912,0.815284,0.458429,0.999829,0.397142,0.827027,0.503398,0.999899,0.396760,...,-0.804045,0.849297,36.167628,84.543863,229.410852,220.959374,26.430373,165.629793,-7.694206,-7.721740


In [64]:
df.shape

(666, 141)

In [65]:
df[df['class'] == 'correct']

,class,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z32,v32,left_elbow_ang,right_elbow_ang,left_lean_ang,right_lean_ang,left_shoulder_ang,right_shoulder_ang,left_elbow_hip_dist,right_elbow_hip_dist
0,correct,0.404247,0.818962,0.455553,0.999754,0.409771,0.828992,0.500907,0.999852,0.408371,...,-0.727213,0.824450,41.644140,162.717884,224.053782,221.801313,22.740362,232.382027,-7.514912,-11.038500
1,correct,0.395100,0.814538,0.459111,0.999775,0.404681,0.828165,0.504184,0.999865,0.403966,...,-0.731567,0.830776,33.426070,172.801361,226.882292,221.880520,6.995678,242.880519,-6.945130,-10.509744
2,correct,0.388115,0.810215,0.461728,0.999794,0.399152,0.826347,0.506682,0.999877,0.399012,...,-0.768031,0.836679,38.467783,121.583713,228.088596,221.279565,13.575731,200.808850,-6.972346,-9.009850
3,correct,0.385210,0.805840,0.463203,0.999813,0.395307,0.822718,0.508092,0.999889,0.395182,...,-0.823461,0.843193,38.201204,81.985306,229.768505,220.880685,18.854596,162.619326,-7.201338,-7.628453
4,correct,0.386912,0.815284,0.458429,0.999829,0.397142,0.827027,0.503398,0.999899,0.396760,...,-0.804045,0.849297,36.167628,84.543863,229.410852,220.959374,26.430373,165.629793,-7.694206,-7.721740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,correct,0.381096,0.716036,-0.221795,0.997948,0.398867,0.734547,-0.180852,0.997797,0.400729,...,0.063348,0.228729,246.148436,245.277741,187.496095,199.729258,3.719560,23.198965,-1.885104,-9.811518
168,correct,0.376995,0.713169,-0.220322,0.998037,0.390329,0.732368,-0.179718,0.997941,0.390803,...,0.035024,0.218820,236.709836,244.616611,188.245606,199.385099,13.782504,28.457155,-5.256188,-12.057450
169,correct,0.370030,0.712660,-0.221814,0.998046,0.384457,0.731992,-0.180537,0.998043,0.385030,...,0.097879,0.209722,241.598496,241.978224,190.458874,203.862814,14.676702,37.088281,-5.820417,-15.915412
170,correct,0.364090,0.709955,-0.215499,0.997862,0.378611,0.729847,-0.177709,0.998004,0.379445,...,0.034388,0.205457,241.014245,115.464297,191.756309,205.348502,32.095759,40.206459,-11.109674,-17.055759


In [66]:
X = df.drop('class', axis= 1) 
y = df['class']

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 7)

In [68]:
y_test

280     error1
384     error2
665     error2
99     correct
485     error2
        ...   
643     error2
82     correct
521     error2
614     error2
664     error2
Name: class, Length: 200, dtype: object

## 3.2 Train Machine Learning Classification Model

In [72]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [73]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [74]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [75]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [76]:
fit_models['rc'].predict(X_test)

array(['error1', 'error2', 'error2', 'correct', 'error2', 'error1',
       'error2', 'error2', 'error2', 'correct', 'error1', 'error2',
       'error2', 'correct', 'error1', 'error2', 'error2', 'error1',
       'error2', 'error1', 'correct', 'error2', 'error2', 'correct',
       'error2', 'error1', 'error2', 'error2', 'correct', 'error2',
       'error2', 'error2', 'error1', 'correct', 'error2', 'error2',
       'error1', 'error2', 'error2', 'correct', 'error2', 'error1',
       'error2', 'error2', 'error2', 'correct', 'error2', 'correct',
       'correct', 'error1', 'correct', 'error2', 'error1', 'error2',
       'error2', 'error1', 'error1', 'error2', 'error2', 'error1',
       'error2', 'error2', 'error1', 'error2', 'error2', 'error2',
       'error1', 'error2', 'error1', 'error2', 'error1', 'error1',
       'error2', 'error1', 'error1', 'error2', 'error2', 'correct',
       'error2', 'error2', 'error2', 'error1', 'correct', 'error2',
       'error1', 'error1', 'error1', 'error1', '

## 3.3 Evaluate and Serialize Model

In [77]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [78]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,
         accuracy_score(y_test.values, yhat),
         precision_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct")
        )

lr 0.995 0.995 0.995 0.995
rc 0.995 0.995 0.995 0.995
rf 0.995 0.995 0.995 0.995
gb 0.99 0.99 0.99 0.99


c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.war

In [79]:
yhat = fit_models['rf'].predict(X_test)

In [80]:
yhat[ :10]

array(['error1', 'error2', 'error2', 'correct', 'error2', 'error1',
       'error2', 'error2', 'error2', 'correct'], dtype=object)

In [81]:
with open(MODEL_PATH, 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [82]:
with open(MODEL_PATH, 'rb') as f:
    pickle.load(f)

In [ ]:
# X = pd.DataFrame([row], columns= landmarks[1:])

In [123]:
def calc(points):
    points.insert(0, "dummy")
    
    # keypoint == keypoint * 4 + 1 (due to zero indentation and action)
    left_shoulder = (points[45], points[46])
    right_shoulder = (points[49], points[50])
    left_elbow = (points[53], points[54])
    right_elbow = (points[57], points[58])
    left_wrist = (points[61], points[62])
    right_wrist = (points[65], points[66])
    left_hip = (points[93], points[94])
    right_hip = (points[97], points[98])
    left_knee = (points[101], points[102])
    right_knee = (points[105], points[106])
    
    left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)
    
    left_lean_ang = angle(left_shoulder, left_hip, left_knee)
    right_lean_ang = angle(right_shoulder, right_hip, right_knee)
    
    left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
    right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)
    
    left_elbow_hip_dist = dist_x(left_hip, left_elbow)
    right_elbow_hip_dist = dist_x(right_hip, right_elbow)
    
    points.append(left_elbow_ang)
    points.append(right_elbow_ang)
    points.append(left_lean_ang)
    points.append(right_lean_ang)
    points.append(left_shoulder_ang)
    points.append(right_shoulder_ang)
    
    points.append(left_elbow_hip_dist)
    points.append(right_elbow_hip_dist)
    return points

In [126]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
state = ''
row = []
# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    while cap.isOpened():
        ret, image = cap.read()
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = pose.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
                
        try:
            row = calc(np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist())
            
            X = pd.DataFrame([row[1:]], columns= landmarks[1:])
            
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)
            correct = False; error1 = False; error2 = False

            if body_language_class == 'correct' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = True
                error1 = False
                error2 = False
            elif body_language_class == 'error1' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = True
                error2 = False
            elif body_language_class == 'error2' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = False
                error2 = True
            
            if (error1):
                state = 'Elbow is out of place'
            elif (correct):
                state = 'Correct'
            elif(error2):
                state = 'Leaning'

            # elif current_stage = 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= .7:
            #     current_stage = 'up'
            #     counter += 1
            
            counter = 0
            
            
                # print(current_stage)

            # Get status box
            #cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)
            
            # Display Class
            cv2.putText(image, 
                        'CLASS',
                        (0,0),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            
            # cv2.putText(image, 
            #             body_language_class(' ')[0],
            #             (90,40),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,
            #             (255,255,255),
            #             2,
            #             cv2.LINE_AA
            #            )
            #Display Propability
            cv2.putText(image, 
                        'PROB',
                        (15,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(round(body_language_prob[np.argmax(body_language_prob)], 2)),
                        (10,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255,255,255),
                        2,
                        cv2.LINE_AA
                       )
            
            #Display Count
            cv2.putText(image, 
                        'COUNT',
                        (180,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(state),
                        (175,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0,0,255),
                        2,
                        cv2.LINE_AA
                       )
            
        except Exception as e:
            pass


        cv2.imshow('Lucio', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

In [120]:
X.shape

(1, 140)